In [1]:
from modules1 import *
from torch.optim import Adam
from  tqdm.notebook import tqdm
import matplotlib.animation as animation
import imageio
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader

In [2]:
batch_size = 275
dataset = torchvision.datasets.FashionMNIST(root="C:/Users/ericy/Downloads", train = True, download = True, transform=torchvision.transforms.ToTensor())
loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [3]:
device = "cuda:1"

model = Unet(time_dim=200, dims=((16, 64),(64,128)), channels=1, device = device)
model = model.to(device)

optimizer = Adam(model.parameters(), lr=1e-5)

In [4]:
time_steps = 200
S = schedule(time_steps, device)

In [ ]:
epochs = 1000
L = []

for epoch in tqdm(range(epochs), desc="Epoch", position=0):
    for step, (batch, _) in enumerate(tqdm(loader, desc="Batch", position=1, leave=False)):
        optimizer.zero_grad()
        batch = batch.to(device)
        t = torch.randint(0, time_steps, (len(batch),), device=device).long()
        loss = S.loss(model, batch, t)
        loss.backward()
        optimizer.step()
        L.append(loss)
        if epoch % 2 == 0 and  step == 0:
            print("Epoch:", epoch, "Loss:", loss.item())

Epoch:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch:   0%|          | 0/219 [00:00<?, ?it/s]

Epoch: 0 Loss: 1.1787360906600952


Batch:   0%|          | 0/219 [00:00<?, ?it/s]

In [ ]:
M = S.sample(model, device, time_steps).astype(np.uint8)
imageio.mimwrite("output2.gif", M, 'GIF', loop=100)